<a href="https://colab.research.google.com/github/jpmantuano/csc612m/blob/main/1D_convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CUDA Programming Project Specifications : 1D Convolution ###

Implements a 1D convolution and compares the performance of CPU-based and GPU-accelerated CUDA implementations, exploring different data transfer methods and CUDA optimization techniques.

#### Implementation of the 1D convolution defined as : out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f ####


```
__global__ void conv1D_kernel(const float* in, float* out, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n - 2) {[link text](https://)
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}
```



#### Input: `2^28` elements or `268435456` ####
It is initialized using the following code:


```
float* initialize_input(float *in, size_t n) {
    for (size_t i = 0; i < n; i++) {
        in[i] = (float)i;
    }
    return in;
}
```



The first and last 20 elements of the output with be displayed at the end of the run, excluding the two zeros at the end.



```
void print_results(const float *out, size_t n) {
    printf("First %d elements of output:\n", PRINT_VALUES);
    for (int i = 0; i < PRINT_VALUES; i++) {
        printf("out[%d] = %f\n", i, out[i]);
    }

    printf("Last %d elements of output:\n", PRINT_VALUES);
    for (size_t i = n - PRINT_VALUES; i < n - 2; i++) {
        printf("out[%zu] = %f\n", i, out[i]);
    }
}
```



The convolution method will be run 30 times and the average runtime will be recorded, this is to accomodate any forms of caching and inconsistencies between runs.



```
//Kernel run method
void run_kernel(const float *in, float *out) {
    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    for (int iter = 0; iter < RUNS; iter++) {
        conv1D_kernel<<<blocks, THREADS_PER_BLOCK>>>(in, out, N);
        cudaDeviceSynchronize();
    }
}
```



Error checking method, also excluding the last two elements as they are always Zero.

```
size_t check_errors(const float *in, const float *out, size_t n) {
    size_t err_count = 0;
    for (size_t i = 0; i < n - 2; i++) {
        float expected = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
        if (fabs(out[i] - expected) > 1e-5) {
            err_count++;
        }
    }
    return err_count;
}
```



###Using C to implement 1D Convolution ###

Get the runtime of 1D Convolution implemented in C and use as the baseline for comparison with the different CUDA version implementation of 1D Convolution.

In [ ]:
%%writefile convolve1D.cpp
#include <iostream>
#include <vector>
#include <ctime>
#include <cstdio>
#include <cmath>

void convolve1D(const std::vector<float>& in, std::vector<float>& out) {
    int n = in.size();
    if (n < 3) {
        out.clear();
        return;
    }
    out.resize(n - 2);
    for (int i = 0; i < n - 2; ++i) {
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}

// Function to check errors between expected and actual output
size_t check_errors(const std::vector<float>& in, const std::vector<float>& out) {
    size_t err_count = 0;
    int n = static_cast<int>(in.size());
    if (out.size() != n - 2) {
        std::cerr << "Output size mismatch: expected " << n - 2 << ", got " << out.size() << std::endl;
        return static_cast<size_t>(-1); // Indicate size mismatch error
    }
    for (int i = 0; i < n - 2; ++i) {
        float expected = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
        if (std::fabs(out[i] - expected) > 1e-5f) {
            err_count++;
        }
    }
    return err_count;
}

int main() {
    int N = 268435456;
    int runs = 30;

    std::vector<float> _in_(N);

    for (int i = 0; i < N; ++i) {
      _in_[i] = static_cast<float>(i + 1);
    }

    std::vector<float> _out_;

    clock_t start, end;
    double cpu_time_used;
    double sum = 0.0;

    for (int run = 0; run < runs; run++) {
      start = clock();  // Start time
      convolve1D(_in_, _out_);
      end = clock(); // End time

      cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
      sum += cpu_time_used;
    }

    // Check for errors
    size_t err_count = check_errors(_in_, _out_);
    if (err_count == static_cast<size_t>(-1)) {
        std::cerr << "Error: Output size mismatch." << std::endl;
    } else if (err_count > 0) {
        std::cerr << "Error count: " << err_count << " mismatches found in output." << std::endl;
    } else {
        std::cout << "Output verification passed: no errors found." << std::endl;
    }

    double avg_time = sum / runs;
    printf("Average execution time for %d inputs: %.6f seconds\n", 2^28, avg_time);

    return 0;
}

Overwriting convolve1D.cpp


In [ ]:
%%shell
g++ -o convolve1D convolve1D.cpp

In [ ]:
%%shell
./convolve1D

Output verification passed: no errors found.
Average execution time for 30 inputs: 3.134053 seconds


### Check if CUDA is present ###

Getting the GPU id for Google Colab but this is often 0 for free tier accounts.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

### Unified memory ###

A single memory address space accessible by both CPU and GPU. [Nvidia Blog](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/)

Key Features

* Simplifies memory management with cudaMallocManaged() instead of separate allocations1.

* Automatic page migration handles data movement between host and device1.

* Eliminates manual cudaMemcpy calls for basic use cases.

Example:

```
float *data;
cudaMallocManaged(&data, N * sizeof(float));  // Accessible by CPU/GPU
```



In [ ]:
%%writefile method_unified_memory.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

#define N 268435456
#define THREADS_PER_BLOCK 1024
#define RUNS 30
#define PRINT_VALUES 20

__global__ void conv1D_kernel(const float* in, float* out, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n - 2) {
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}

void initialize_input(float *in, size_t n) {
    for (size_t i = 0; i < n; i++) {
        in[i] = (float)i;
    }
}

void run_kernel_multiple_times(const float *in, float *out) {
    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    for (int iter = 0; iter < RUNS; iter++) {
        conv1D_kernel<<<blocks, THREADS_PER_BLOCK>>>(in, out, N);
        cudaDeviceSynchronize();
    }
}

void print_results(const float *out, size_t n) {
    printf("First %d elements of output:\n", PRINT_VALUES);
    for (int i = 0; i < PRINT_VALUES; i++) {
        printf("out[%d] = %f\n", i, out[i]);
    }

    printf("Last %d elements of output:\n", PRINT_VALUES);
    for (size_t i = N - 20; i < N - 2; i++) {
        printf("out[%zu] = %f\n", i, out[i]);
    }
}

size_t check_errors(const float *in, const float *out, size_t n) {
    size_t err_count = 0;
    for (size_t i = 0; i < n - 2; i++) {
        float expected = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
        if (fabs(out[i] - expected) > 1e-5) {
            err_count++;
        }
    }
    return err_count;
}

void method_unified_memory(float *in, float *out, size_t n) {
    run_kernel_multiple_times(in, out);

    print_results(out, n);

    size_t err_count = check_errors(in, out, n);
    printf("Error count(CUDA program): %zu\n", err_count);
}

int main() {
    float *in = NULL;
    float *out = NULL;

    cudaMallocManaged(&in, N * sizeof(float));
    cudaMallocManaged(&out, N * sizeof(float));

    initialize_input(in, N);

    method_unified_memory(in, out, N);

    cudaFree(in);
    cudaFree(out);

    printf("\n");
    return 0;
}


In [ ]:
%%shell
nvcc method_unified_memory.cu -o method_unified_memory -arch=sm_75

In [ ]:
%%shell
./method_unified_memory

In [ ]:
%%shell
nvprof ./method_unified_memory

### Prefetching data with Memory Advice ###

Explicit control over data locality using:

**a. cudaMemPrefetchAsync**

Pre-migrates data to a specific processor (CPU/GPU) before access:

```
cudaMemPrefetchAsync(data, size, deviceId);  // Force data to GPU

```
**b. cudaMemAdvise**

Hints about access patterns:
```
cudaMemAdvise(data, size, cudaMemAdviseSetAccessedBy, deviceId);  // GPU will access

```

In [ ]:
%%writefile method_prefetching_memory_advice.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

#define N 268435456
#define THREADS_PER_BLOCK 1024
#define RUNS 30
#define VALUES 20

__global__ void conv1D_kernel(const float* in, float* out, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n - 2) {
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}

void allocate_managed_memory(float **in, float **out) {
    cudaMallocManaged(in, N * sizeof(float));
    cudaMallocManaged(out, N * sizeof(float));
}

void set_memory_advice(float *in, float *out) {
    cudaMemAdvise(in, N * sizeof(float), cudaMemAdviseSetPreferredLocation, 0);
    cudaMemAdvise(out, N * sizeof(float), cudaMemAdviseSetPreferredLocation, 0);
}

void initialize_input(float *in) {
    for (size_t i = 0; i < N; i++) {
        in[i] = (float)i;
    }
}

void prefetch_to_device(float *in, float *out) {
    cudaMemPrefetchAsync(in, N * sizeof(float), 0);
    cudaMemPrefetchAsync(out, N * sizeof(float), 0);
    cudaDeviceSynchronize();
}

void run_kernel_multiple_times(const float *in, float *out) {
    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    for (int iter = 0; iter < RUNS; iter++) {
        conv1D_kernel<<<blocks, THREADS_PER_BLOCK>>>(in, out, N);
        cudaDeviceSynchronize();
    }
}

void prefetch_to_host(float *out) {
    cudaMemPrefetchAsync(out, N * sizeof(float), cudaCpuDeviceId);
    cudaDeviceSynchronize();
}

void print_results(const float *out) {
    printf("First %d elements of output:\n", VALUES);
    for (int i = 0; i < VALUES; i++) {
        printf("out[%d] = %f\n", i, out[i]);
    }

    printf("Last %d elements of output:\n", VALUES);
    for (int i = N - VALUES; i < N - 2; i++) {
        printf("out[%d] = %f\n", i, out[i]);
    }
}

void check_errors(const float *in, const float *out) {
    size_t err_count = 0;
    for (size_t i = 0; i < N-2; i++) {
        float expected = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
        if (fabs(out[i] - expected) > 1e-5) {
            err_count++;
        }
    }
    printf("Error count(CUDA program): %zu\n", err_count);
}

void cleanup(float *in, float *out) {
    cudaFree(in);
    cudaFree(out);
}

void method_prefetching_memory_advice() {
    float *in = NULL, *out = NULL;

    allocate_managed_memory(&in, &out);
    set_memory_advice(in, out);
    initialize_input(in);
    prefetch_to_device(in, out);
    run_kernel_multiple_times(in, out);
    prefetch_to_host(out);
    print_results(out);
    check_errors(in, out);
    cleanup(in, out);
}

int main() {
    method_prefetching_memory_advice();
    printf("\n");
    return 0;
}


In [ ]:
%%shell
nvcc method_prefetching_memory_advice.cu -o method_prefetching_memory_advice -arch=sm_75

In [ ]:
%%shell
./method_prefetching_memory_advice

In [ ]:
%%shell
nvprof ./method_prefetching_memory_advice

### Data initialization as a CUDA kernel ###

Direct GPU-side initialization avoids host-device transfers:

**Advantages**

* Eliminates cudaMemcpy for initial data setup.

* Faster for large datasets (no PCIe transfer).

Example:
```
__global__ void init_kernel(float *data, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) data[i] = i;  // Initialize directly on GPU
}
```

In [ ]:
%%writefile method_init_data_kernel.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

#define N 268435456
#define THREADS_PER_BLOCK 1024
#define RUNS 30
#define VALUES 20

// CUDA kernel: 1D convolution averaging 3 elements
__global__ void conv1D_kernel(const float* in, float* out, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n - 2) {
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}

// CUDA kernel: initialize data on device with index values
__global__ void init_data_kernel(float* data, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        data[i] = (float)i;
    }
}

// Allocate unified memory for input and output arrays
void allocate_memory(float** in, float** out, size_t size) {
    cudaMallocManaged((void**)in, size);
    cudaMallocManaged((void**)out, size);
}

// Free allocated unified memory
void free_memory(float* in, float* out) {
    cudaFree(in);
    cudaFree(out);
}

// Launch kernel to initialize input data on GPU
void initialize_data(float* in, int n, int blocks) {
    init_data_kernel<<<blocks, THREADS_PER_BLOCK>>>(in, n);
    cudaDeviceSynchronize();
}

// Launch kernel to perform 1D convolution
void perform_convolution(const float* in, float* out, int n, int blocks) {
    for (int iter = 0; iter < RUNS; iter++) {
        conv1D_kernel<<<blocks, THREADS_PER_BLOCK>>>(in, out, N);
        cudaDeviceSynchronize();
    }
}

// Print first and last 20 elements of output array
void print_results(const float* out, int n) {
    int i;
    printf("First 20 elements of output:\n");
    for (i = 0; i < 20; i++) {
        printf("out[%d] = %f\n", i, out[i]);
    }

    printf("Last 20 elements of output:\n");
    for (i = n - 20; i < n - 2; i++) {
        printf("out[%d] = %f\n", i, out[i]);
    }
}

// Check for errors comparing output with expected values
size_t check_errors(const float* in, const float* out, int n) {
    size_t err_count = 0;
    int i;
    for (i = 0; i < n - 2; i++) {
        float expected = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
        if (fabsf(out[i] - expected) > 1e-5f) {
            err_count++;
        }
    }
    return err_count;
}

void method_init_data_kernel() {
    float *in = NULL, *out = NULL;
    size_t size = N * sizeof(float);
    allocate_memory(&in, &out, size);

    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    initialize_data(in, N, blocks);
    perform_convolution(in, out, N, blocks);

    print_results(out, N);

    size_t err_count = check_errors(in, out, N);
    printf("Error count (CUDA program): %zu\n", err_count);

    free_memory(in, out);
}

int main() {
    method_init_data_kernel();
    printf("\n");
    return 0;
}

In [ ]:
%%shell
nvcc method_init_data_kernel.cu -o method_init_data_kernel -arch=sm_75

In [ ]:
%%shell
nvprof ./method_init_data_kernel

In [ ]:
%%shell
./method_init_data_kernel

### Old method of data transfer ###

Explicit host/device memory management:

**Steps**

1. Allocate host memory (malloc).
2. Allocate device memory (cudaMalloc).
3. Copy data:
```
cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);  // Host→Device[6][7]
```
4. Process on GPU.
5. Copy results back:
```
cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);  // Device→Host[6][7]
```





In [ ]:
%%writefile method_old_transfer.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>

#define N 268435456
#define THREADS_PER_BLOCK 1024

// CUDA kernel: 1D convolution averaging 3 elements
__global__ void conv1D_kernel(const float* in, float* out, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n - 2) {
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}

// Allocate host memory for input and output arrays
void allocate_host_memory(float** h_in, float** h_out, size_t size) {
    *h_in = (float*)malloc(size);
    *h_out = (float*)malloc(size);
    if (*h_in == NULL || *h_out == NULL) {
        fprintf(stderr, "Failed to allocate host memory\n");
        exit(EXIT_FAILURE);
    }
}

// Free host memory
void free_host_memory(float* h_in, float* h_out) {
    free(h_in);
    free(h_out);
}

// Initialize input data on host
void initialize_host_data(float* h_in, int n) {
    int i;
    for (i = 0; i < n; i++) {
        h_in[i] = (float)i;
    }
}

// Allocate device memory for input and output arrays
void allocate_device_memory(float** d_in, float** d_out, size_t size) {
    cudaError_t err;
    err = cudaMalloc((void**)d_in, size);
    if (err != cudaSuccess) {
        fprintf(stderr, "cudaMalloc d_in failed: %s\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
    err = cudaMalloc((void**)d_out, size);
    if (err != cudaSuccess) {
        fprintf(stderr, "cudaMalloc d_out failed: %s\n", cudaGetErrorString(err));
        cudaFree(*d_in);
        exit(EXIT_FAILURE);
    }
}

// Free device memory
void free_device_memory(float* d_in, float* d_out) {
    cudaFree(d_in);
    cudaFree(d_out);
}

// Copy data from host to device
void copy_host_to_device(float* d_in, float* h_in, size_t size) {
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);
}

// Copy data from device to host
void copy_device_to_host(float* h_out, float* d_out, size_t size) {
    cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);
}

// Launch convolution kernel
void launch_convolution_kernel(const float* d_in, float* d_out, int n, int blocks) {
    for (int iter = 0; iter < RUNS; iter++) {
      conv1D_kernel<<<blocks, THREADS_PER_BLOCK>>>(d_in, d_out, n);
      cudaDeviceSynchronize();
    }
}

// Print first and last 20 elements of output array
void print_results(const float* h_out, int n) {
    int i;
    printf("First 20 elements of output:\n");
    for (i = 0; i < 20; i++) {
        printf("h_out[%d] = %f\n", i, h_out[i]);
    }

    printf("Last 20 elements of output:\n");
    for (i = n - 20; i < n - 2; i++) {
        printf("h_out[%d] = %f\n", i, h_out[i]);
    }
}

// Check for errors comparing output with expected values
size_t check_errors(const float* h_in, const float* h_out, int n) {
    size_t err_count = 0;
    int i;
    for (i = 0; i < n - 2; i++) {
        float expected = (h_in[i] + h_in[i + 1] + h_in[i + 2]) / 3.0f;
        if (fabsf(h_out[i] - expected) > 1e-5f) {
            err_count++;
        }
    }
    return err_count;
}

// Main method encapsulating the workflow
void method_old_transfer() {
    printf("Method 4: Old Method with cudaMalloc + cudaMemcpy\n");

    float *h_in = NULL, *h_out = NULL;
    float *d_in = NULL, *d_out = NULL;
    size_t size = N * sizeof(float);

    allocate_host_memory(&h_in, &h_out, size);
    initialize_host_data(h_in, N);
    allocate_device_memory(&d_in, &d_out, size);

    copy_host_to_device(d_in, h_in, size);

    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    launch_convolution_kernel(d_in, d_out, N, blocks);

    copy_device_to_host(h_out, d_out, size);

    print_results(h_out, N);

    size_t err_count = check_errors(h_in, h_out, N);
    printf("Error count (CUDA program): %zu\n", err_count);

    free_device_memory(d_in, d_out);
    free_host_memory(h_in, h_out);
}

int main() {
    method_old_transfer();
    return 0;
}


In [ ]:
%%shell
nvcc method_old_transfer.cu -o method_old_transfer -arch=sm_75

In [ ]:
%%shell
nvprof ./method_old_transfer